In [139]:
import os
import re
from pathlib import Path
import pandas as pd
import glob
import tiktoken
import statistics

## Directories and files

In [132]:
directory = Path("../text/KMGanguli")
out_directory = Path("../data")
source = "http://100words.lauragibbs.net/"
file_list = glob.glob(f"{directory}/*.txt")
file_list.sort()


In [133]:
rows = []


for file in file_list:
    book_number = int(file.split("/")[-1].replace(".txt", ""))
    
    # print(book_number)
    current_section = "NA"
    current_section_name = "NA"
    text = ""

    with open(file) as book:
        for line in book:
            if line.strip()=="":
                # print('Row complete')
                rows = rows + [{'book_number': book_number, 'section': current_section, 'section_name': current_section_name, 'text': text}]
                text = ""
                continue

            if re.match('^(SECTION) .+$', line) or re.match('^\d+ *$', line):
                # print(line)
                current_section = line.strip()
                current_section_name = ""
                continue

            if re.match("^\(.+(Parva).*\).*$",  line, flags=re.DOTALL):
                # print(line)
                current_section_name = line.strip().replace("(", "").replace(")", "").replace("\n", "")
                continue
            
            text = text + line


In [136]:
df_kng = pd.DataFrame(rows)


In [140]:
encoder_name = "cl100k_base"
encoding = tiktoken.get_encoding(encoder_name)
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

## Calculat the text tokens per row
df_kng['num_text_tokens'] = df_kng['text'].apply(num_tokens_from_string)

statistics.mean(df_kng['num_text_tokens'])

207.2837941628264

In [138]:
df_kng.to_csv(out_directory/"kn_ganguli_translation.csv", index=False, sep="|")